Part 2: Depth from Rectified Stereo Images

Write a program to estimate a depth map from the pair of rectified images in Figure 2 using an MRF and multiple label graphcuts.
Show your best result and provide some discussion if necessary (particularly if the results are not as good as expected).

In [ ]:

import numpy
from gco import pygco
from utils import get_logger, INTEGER_MAX_VALUE
import matplotlib.image
import matplotlib.pyplot

In [ ]:
def compute_distance(point1: numpy.ndarray, point2: numpy.ndarray) -> int:
    """
    compute distance of two pixels
    Parameters
    ----------
    point1: ndarray, uint8, shape=(3)
        point one
    point2: ndarray, uint8, shape=(3)
        point two
    """
    # return (abs(point1[0]-point2[0]) + abs(point1[1] - point2[1]) + abs(point1[2] - point2[2]) ) / 3 / 255
    # if not dividing 255 as the C pseudocode example, there will be a overflowing of MAX_VALUE of Integer when calling pycgo.cut_grid_graph
    return numpy.sum(numpy.abs(point1-point2)) // 3